In [1]:
import requests
import time
import os
import cv2
import numpy as np

def save_image_from_url(url, folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    try:
        while True:
            response = requests.get(url)
            if response.status_code == 200:
                timestamp = time.strftime("%Y%m%d-%H%M%S")
                filename = os.path.join(folder_path, f"{timestamp}.jpg")
                
                # Convert the image bytes to a numpy array
                image_np = np.frombuffer(response.content, dtype=np.uint8)
                
                # Decode the numpy array to an image
                image = cv2.imdecode(image_np, flags=cv2.IMREAD_COLOR)
                
                # Resize the image to the desired size
                resized_image = cv2.resize(image, (1280, 720))

                # Save the resized image to file
                cv2.imwrite(filename, resized_image)
            else:
                print(f"Failed to download image. Status code: {response.status_code}")
            
            time.sleep(5)  # Wait for 5 seconds
    except KeyboardInterrupt:
        print("Terminated by user.")

# Example usage:
save_image_from_url("https://livecamera.city.kashiwa.lg.jp/-wvhttp-01-/image.cgi?1691293292249", 'camera')

Terminated by user.


In [2]:
import cv2
import csv
import os

def count_cars_in_image(image_path, bg_model):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    # Convert to grayscale for processing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Compute the absolute difference between the current image and background model
    fgmask = bg_model.apply(gray)

    # Apply thresholding to eliminate noise
    _, thresh = cv2.threshold(fgmask, 25, 255, cv2.THRESH_BINARY)

    # Dilation to merge adjacent white regions
    dilated = cv2.dilate(thresh, None, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    valid_contours = [contour for contour in contours if cv2.contourArea(contour) > 500]

    return len(valid_contours)

def process_images_and_save_to_csv(folder_path, csv_path):
    bg_model = cv2.createBackgroundSubtractorMOG2(history=5, varThreshold=100, detectShadows=True)

    with open(csv_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Name", "Car Count"])

        for image_name in sorted(os.listdir(folder_path)):
            image_path = os.path.join(folder_path, image_name)
            if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                car_count = count_cars_in_image(image_path, bg_model)
                writer.writerow([image_name, car_count])

# Example usage
process_images_and_save_to_csv('camera', 'car_counts.csv')

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import csv

def load_model():
    # Load a pre-trained model from TensorFlow Hub
    model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
    return model

def count_cars_in_image(image_path, model):
    image = tf.image.decode_image(tf.io.read_file(image_path))
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, dtype=tf.uint8)  # Cast to uint8
    image = tf.expand_dims(image, 0)

    detections = model(image)
    class_ids = detections["detection_classes"][0].numpy().astype(int)
    scores = detections["detection_scores"][0].numpy()

    # Assuming class ID 3 is for cars (based on COCO dataset)
    car_detections = sum(1 for class_id, score in zip(class_ids, scores) if class_id == 3 and score > 0.5)
    return car_detections

def process_images_and_save_to_csv(folder_path, csv_path):
    model = load_model()

    with open(csv_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Name", "Car Count"])

        for image_name in sorted(os.listdir(folder_path)):
            image_path = os.path.join(folder_path, image_name)
            if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                car_count = count_cars_in_image(image_path, model)
                writer.writerow([image_name, car_count])

# Example usage
process_images_and_save_to_csv('camera', 'car_counts.csv')

ModuleNotFoundError: No module named 'tensorflow.python'

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import csv
import cv2
import numpy as np

def load_model():
    model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
    return model

def count_cars_in_image(image_path, model, display=False):
    image = tf.image.decode_image(tf.io.read_file(image_path))
    original_image = image.numpy().astype(np.uint8)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, dtype=tf.uint8)
    image = tf.expand_dims(image, 0)

    detections = model(image)
    class_ids = detections["detection_classes"][0].numpy().astype(int)
    scores = detections["detection_scores"][0].numpy()
    boxes = detections["detection_boxes"][0].numpy()  # Bounding boxes

    car_detections = 0
    for class_id, score, box in zip(class_ids, scores, boxes):
        if class_id == 3 and score > 0.5:
            car_detections += 1
            if display:
                # Convert box coordinates to pixel coordinates for original image
                height, width, _ = original_image.shape
                ymin, xmin, ymax, xmax = box
                start_point = (int(xmin * width), int(ymin * height))
                end_point = (int(xmax * width), int(ymax * height))
                cv2.rectangle(original_image, start_point, end_point, (255, 0, 0), 2)

    if display:
        cv2.imshow("Detected Cars", original_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return car_detections

def process_images_and_save_to_csv(folder_path, csv_path):
    model = load_model()

    with open(csv_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Name", "Car Count"])

        for image_name in sorted(os.listdir(folder_path)):
            image_path = os.path.join(folder_path, image_name)
            if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                car_count = count_cars_in_image(image_path, model, display=True)  # Set display to True to see the images
                writer.writerow([image_name, car_count])

process_images_and_save_to_csv('camera', 'car_counts.csv')

ModuleNotFoundError: No module named 'tensorflow.python'

In [5]:
import sys
from pathlib import Path
import cv2
import csv
import os

# git clone した pytorch_yolov3 ディレクトリのパスを指定してください
yolov3_path = Path("../pytorch_yolov3")

sys.path.append(str(yolov3_path))
from yolov3.detector import Detector

config_path = yolov3_path / "config/yolov3_coco.yaml"
weights_path = yolov3_path / "weights/yolov3.weights"

# 検出器を作成する。
detector = Detector(config_path, weights_path)

def count_cars_in_image(image_path, display=False):
    # 画像を読み込む。
    img = cv2.imread(image_path)

    # 検出する。
    detections = detector.detect_from_imgs(img)

    # 車の検出結果のみ抽出する。
    target = ["bicycle", "car", "motorcycle", "bus", "truck"]
    cars = list(filter(lambda x: x["class_name"] in target, detections[0]))

    if display:
        for bbox in cars:
            cv2.rectangle(
                img,
                (int(bbox["x1"]), int(bbox["y1"])),
                (int(bbox["x2"]), int(bbox["y2"])),
                color=(0, 255, 0),
                thickness=2,
            )
        cv2.imshow("Detected Cars", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return len(cars)

def process_images_and_save_to_csv(folder_path, csv_path):
    with open(csv_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Name", "Car Count"])

        for image_name in sorted(os.listdir(folder_path)):
            image_path = os.path.join(folder_path, image_name)
            if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                car_count = count_cars_in_image(image_path, display=True)  # Set display to True to see the images
                writer.writerow([image_name, car_count])

# Example usage
process_images_and_save_to_csv('camera', 'car_counts.csv')

ModuleNotFoundError: No module named 'yolov3'

In [1]:
import sys
from pathlib import Path
import cv2
import csv
import os

# git clone した pytorch_yolov3 ディレクトリのパスを指定してください
yolov3_path = Path("../pytorch_yolov3")

sys.path.append(str(yolov3_path))
from yolov3.detector import Detector

config_path = yolov3_path / "config/yolov3_coco.yaml"
weights_path = yolov3_path / "weights/yolov3.weights"

# 検出器を作成する。
detector = Detector(config_path, weights_path)

def count_cars_in_image(image_path, display=False):
    # 画像を読み込む。
    img = cv2.imread(image_path)

    # 検出する。
    detections = detector.detect_from_imgs(img)

    # 車の検出結果のみ抽出する。
    target = ["bicycle", "car", "motorcycle", "bus", "truck"]
    cars = list(filter(lambda x: x["class_name"] in target, detections[0]))

    if display:
        for bbox in cars:
            cv2.rectangle(
                img,
                (int(bbox["x1"]), int(bbox["y1"])),
                (int(bbox["x2"]), int(bbox["y2"])),
                color=(0, 255, 0),
                thickness=2,
            )
            label = f"{bbox['class_name']} {bbox['confidence']:.2f}"
            cv2.putText(img, label, (int(bbox["x1"]), int(bbox["y1"]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

        cv2.imshow("Detected Cars", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return len(cars)

def process_images_and_save_to_csv(folder_path, csv_path):
    with open(csv_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Name", "Car Count"])

        for image_name in sorted(os.listdir(folder_path)):
            image_path = os.path.join(folder_path, image_name)
            if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                car_count = count_cars_in_image(image_path, display=True)  # Set display to True to see the images
                writer.writerow([image_name, car_count])

# Example usage
process_images_and_save_to_csv('camera', 'car_counts.csv')

Darknet format weights file loaded. ..\pytorch_yolov3\weights\yolov3.weights
